# Analysing data on The Math Geaneology project
https://www.genealogy.math.ndsu.nodak.edu/

## Data

add link to data.json

## Reading data

In [ ]:
import json
import pandas as pd

In [ ]:
with open("data/data.json", "r") as f:
    data = json.load(f)
data=data["nodes"]
data=pd.DataFrame(data)
data.head()

## Fetching coordinates of schools

We use OpenStreetMaps to get the coordinates of the schools

In [ ]:
import time

# Using Bing maps API for better performance. This requires a free account and an API key
from geopy.geocoders import Bing

# Alternatively we can use OpenStreetMaps API which is free and open source, but does not perform as well as Bing
#from geopy.geocoders import Nominatim

Defining the `get_coordinates` function using geopy package

In [ ]:
def get_coordinates(university_name):

    # Bing API Account
    api_key=...
    geolocator = Bing(api_key=api_key)

    # Initialize the Nominatim geocoder
    #geolocator = Nominatim(user_agent="university_locator")

    university_name_no_filler = remove_fillers(university_name)

    try:
        # Use the geocode method to get the location information
        location = geolocator.geocode(university_name_no_filler)

        if location:
            # Extract latitude and longitude
            latitude, longitude = location.latitude, location.longitude
            return [university_name,latitude,longitude]
            
        else:
            # if we did not get the location return location in Antartica
            return [university_name,-82,135]
            #print(f"Coordinates for {university_name} not found.")

    except Exception as e:
        #print(f"An error occurred: {e}")
        #If there is an error we return None
        return [university_name,None,None]

def remove_fillers(sentence):

    sentence = str(sentence)
    # List of filler words
    filler_words = ['the', 'of', 'and', 'in', 'to', 'a', 'is', 'that', 'it', 'with', 'as', 'on', 'for', 'at']

    # Split the sentence into words
    words = sentence.split()

    # Remove filler words
    filtered_words = [word for word in words if word.lower() not in filler_words]

    # Join the remaining words to form the new sentence
    new_sentence = ' '.join(filtered_words)

    return new_sentence

In [ ]:
import csv
from tqdm import tqdm

# Get the unique schools
schools = data['school'].unique()

# Loop through the schools and save the recieved coordinates in coord
coords =[]
count = 0
for school in tqdm(schools):
    coord = get_coordinates(school)
    coords.append(coord)
    count+=1
    #if count%50==0:
        #print(count)

    # Save coordinates every 500 iterations
    if count%500==0:
        with open('data/coords'+str(count)+'.csv','w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerows(coords)

    time.sleep(0.2)

# Save all the coordinates
with open('data/coords'+str(count)+'.csv','w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerows(coords)

print("Done")